In [ ]:
#Project: Import csv to bigquery table
#StartDate: 4/06/2022
#EndDate: 
#Developer: Bradley, Hongquy, Khoa

In [1]:
from flask import Flask,render_template,request,redirect,url_for
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage
import os, re, datetime, uuid, time, json
import google.cloud.logging
import apache_beam as beam
from io import StringIO
import argparse
from apache_beam.options.pipeline_options import PipelineOptions
from werkzeug.utils import secure_filename

In [2]:
#this is to allow a authenticated user on the project to work on bigquerry
Key_path = r"C:\\Users\Brad\Documents\model-craft-342921-ea36cdb339e7.json"

#this is needed to request a job
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = Key_path
credentials = service_account.Credentials.from_service_account_file(Key_path,scopes=["https://www.googleapis.com/auth/cloud-platform"])
client=bigquery.Client(credentials=credentials,project=credentials.project_id)
#table_id='model-craft-342921.testing.Task3'
storage_client=storage.Client(credentials=credentials,project=credentials.project_id)
bucket = storage_client.get_bucket("data_intake4062022")
bucket1 = storage_client.get_bucket("error_messages4142022")
bucket2 = storage_client.get_bucket("practice_error_logs")
client1 = google.cloud.logging.Client()
logger = client1.logger(name="dataflow.googleapis.com%2Fworker")

In [ ]:
from io import StringIO
import json, re
blob = bucket2.blob('LOG: 2022-04-15 15:48:41.064521.txt')
downloaded_blob = blob.download_as_string()
log_message = downloaded_blob.decode("utf-8", "ignore")
log_message =log_message.split('\n')
bad_row = []
bad_json = []
result = {}
for element in log_message:
    element = element.replace("'model-craft-342921:testing.Task3', ",'')
    element = element.replace('(','')
    element = element.replace(')','')
    element = element.replace("'",'"')
    bad_row.append(element)
for index in range(0,len(bad_row)-1):
    bad_json.append(json.loads(bad_row[index]))
result['elements'] = bad_json

In [ ]:
from io import StringIO
import json, re
blob = bucket1.blob('LOG: 2022-04-15 15:48:41.064521.txt')
downloaded_blob = blob.download_as_string()
log_message = downloaded_blob.decode("utf-8", "ignore")
log_message = log_message.replace("'",'"')
bad_message = json.loads(log_message)
result1 = {}
result1['elements'] = bad_message

In [ ]:
final_bad_rows = []
count = 0
for rec in result1['elements']:
    for input_data in result['elements']:
        message = ''
        if input_data[rec['location']] == rec['message'].split(': ')[1]:
            message = 'Error #{} - Message: {} in column {} - Input: {}'.format(count, rec['message'],rec['location'],input_data)
            count += 1
            result['elements'].remove(input_data)
            final_bad_rows.append(message)

            
for p in final_bad_rows:
    print(p)

filter_str = (
    f'resource.labels.job_name=pythontobigquerry-26828c6d-9c18-4fd4-a010-aec4cd47f78f'
    f' resource.type="dataflow_step"'
    f' AND severity >= ERROR'
)
messages = []
log_builder = {}
for entry in logger.list_entries(filter_=filter_str):  # API call(s)
    message = entry.to_api_repr()['jsonPayload']['message']
    list_extract = message.replace("There were errors inserting to BigQuery. Will not retry. Errors were ","")
    list_extract = list_extract.replace("'",'"')
    res = json.loads(list_extract)
    for i in range(0,len(res)):
        log_builder = {}
        log_builder['location'] = res[i]['errors'][0]['location']
        log_builder['message'] = res[i]['errors'][0]['message']
        messages.append(log_builder)
print(messages)

In [ ]:
class formating(beam.DoFn):
    def process(self, element, table_sch):
        import apache_beam as beam
        record = {}
        for x in range(0, len(element)):
            name = table_sch[x]['name']
            record[name] = element[x]
        yield record

In [ ]:
class process_message(beam.DoFn):
    def process(self, element, bad_input_data):
        import apache_beam as beam
        from io import StringIO
        import json, re
        count = 0
        for input_data in bad_input_data['elements']:
            message = ''
            if input_data[element['location']] == element['message'].split(': ')[1]:
                message = 'Error #{} - Message: {} in column {} - Input: {}'.format(count, element['message'],element['location'],input_data)
                count += 1
                bad_input_data['elements'].remove(input_data)
                yield message

In [3]:
filename = 'LOG: 2022-04-15 15:48:41.064521.txt'
job_name = 'pythontobigquerry-{}'.format(uuid.uuid4())
beam_options = PipelineOptions(
                            runner = 'DataflowRunner',
                            #runner='DirectRunner',
                            project = 'model-craft-342921',
                            job_name = '{}'.format(job_name),
                            temp_location = 'gs://data_intake4062022/temp1',
                            region='us-east1',
                            service_account_email = 'practice-py@model-craft-342921.iam.gserviceaccount.com'
                        )
try:
    p1 = beam.Pipeline(options=beam_options)
    events  = (p1 | 'ReadData' >> beam.io.ReadFromText('gs://practice_error_logs/{}'.format(filename))
                    | 'Store Final Message' >> beam.io.WriteToText('gs://complete_message_4182022/{}'.format(filename), shard_name_template = "")
              )
    result = p1.run()
    result.wait_until_finish()

except Exception as error:
    print('This was the error: ', error)

In [ ]:
filename = 'LOG: 2022-04-15 15:48:41.064521.txt'
job_name = 'pythontobigquerry-{}'.format(uuid.uuid4())
try:
    blob = bucket2.blob(filename)
    downloaded_blob = blob.download_as_string()
    log_message = downloaded_blob.decode("utf-8", "ignore")
    log_message =log_message.split('\n')
    bad_row = []
    bad_json = []
    bad_input_data = {}
    for element in log_message:
        element = element.replace("'model-craft-342921:testing.Task3', ",'')
        element = element.replace('(','')
        element = element.replace(')','')
        element = element.replace("'",'"')
        bad_row.append(element)
    for index in range(0,len(bad_row)-1):
        bad_json.append(json.loads(bad_row[index]))
    bad_input_data['elements'] = bad_json

except Exception as error:
    print('This was the error: ', error)
    message = "Error setting up messaging"
    
beam_options = PipelineOptions(
                            runner = 'DataflowRunner',
                            #runner='DirectRunner',
                            project = 'model-craft-342921',
                            job_name = '{}'.format(job_name),
                            temp_location = 'gs://data_intake4062022/temp1',
                            region='us-east1',
                            service_account_email = 'practice-py@model-craft-342921.iam.gserviceaccount.com'
                        )
try:
    p1 = beam.Pipeline(options=beam_options)
    events  = (p1 | 'ReadData' >> beam.io.ReadFromText('gs://error_messages4142022/{}'.format(filename))
                  | 'Process Messaging' >> beam.ParDo(process_message(),bad_input_data)
                    | 'Store Final Message' >> beam.io.WriteToText('gs://complete_message_4182022/{}'.format(filename), shard_name_template = "")
              )
    result = p1.run()
    result.wait_until_finish()

except Exception as error:
    print('This was the error: ', error)

In [ ]:
#this is needed to run the website html
app = Flask(__name__)

#secret key is to allow this python code to move variable from the frontend to the backend
app.secret_key = "35367565"

@app.route('/')
def form():
    return redirect(url_for('upload'))

#This will render the frontend of the website to get the json file
@app.route('/upload', methods=['GET','POST'])
def upload():

    #resets the message so that previous messages dont confuse the user
    message = None
    
    tables = ['Task3'] #change--------------------------
    
    job_name = 'pythontobigquerry-{}'.format(uuid.uuid4())

    beam_options = PipelineOptions(
                                runner = 'DataflowRunner',
                                #runner='DirectRunner',
                                project = 'model-craft-342921',
                                job_name = '{}'.format(job_name),
                                temp_location = 'gs://data_intake4062022/temp1',
                                region='us-east1',
                                maxNumWorkers= 1,
                                service_account_email = 'practice-py@model-craft-342921.iam.gserviceaccount.com'
                            )
    filter_str = (
        f'resource.labels.job_name={job_name}'
        f' resource.type="dataflow_step"'
        f' AND severity >= ERROR'
    )
    
    
    #This will only run if the user attempts to submit a file
    if request.method == 'POST':

        #this will only run if what the user submitted is a file
        if 'file' in request.files:
            #this gets the file data
            file = request.files['file']
            #this aquires the name of the file
            filename = secure_filename(file.filename)
            if len(file.readlines()) == 0:
                message = "File has no data to process"
            else:
                file.seek(0)
                try:
                    #this will only run if the file is a csv
                    if filename.endswith('.csv'):
                        blob = bucket.blob('data.csv')
                        blob.upload_from_file(file)
                        
                        file.seek(0)
                        total_records = len(file.readlines())
                        
                        table_id = request.form.getlist('checks')[0]
                        SCHEMA = {}
                        table_sch = []
                        try:
                            SchemaJob = client.get_table('model-craft-342921.testing.{}'.format(table_id))
                            for s in SchemaJob.schema:
                                new_dict = {}
                                new_dict['name'] = s.name
                                new_dict['type'] = s.field_type
                                new_dict['mode'] = s.mode
                                table_sch.append(new_dict)
                            SCHEMA['fields'] = table_sch

                        except Exception as e:
                            print(e)
                        filename_helper = str(datetime.datetime.now())
                        filename = "LOG: "+ filename_helper + ".txt"
                        try:
                            #start = time.time()
                            p = beam.Pipeline(options=beam_options)
                            events  = (p | 'ReadData' >> beam.io.ReadFromText('gs://data_intake4062022/data.csv', skip_header_lines =1)
                                   | 'Split' >> beam.Map(lambda x: x.split(','))
                                    | 'format to dict2' >> beam.ParDo(formating(),table_sch) 
                                   | 'WriteToBigQuery2' >>  beam.io.gcp.bigquery.WriteToBigQuery(
                                       'model-craft-342921:testing.{}'.format(table_id),
                                       schema=SCHEMA,
                                       write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
                                       insert_retry_strategy=beam.io.gcp.bigquery_tools.RetryStrategy.RETRY_NEVER,
                                       method='STREAMING_INSERTS')
                                    )
                            (events[beam.io.gcp.bigquery.BigQueryWriteFn.FAILED_ROWS]
                                    | "Bad lines" >> beam.io.WriteToText('gs://practice_error_logs/{}'.format(filename), shard_name_template = "")
                            )
                            
                            result = p.run()
                            result.wait_until_finish()
                            
                        except Exception as error:
                            print('This was the error: ', error)
                            message = "Error setting up the data ingestion pipeline"
                            
                        try:
                            messages = []
                            log_builder = {}
                            for entry in logger.list_entries(filter_=filter_str):  # API call(s)
                                message = entry.to_api_repr()['jsonPayload']['message']
                                list_extract = message.replace("There were errors inserting to BigQuery. Will not retry. Errors were ","")
                                list_extract = list_extract.replace("'",'"')
                                res = json.loads(list_extract)
                                for i in range(0,len(res)):
                                    log_builder = {}
                                    log_builder['location'] = res[i]['errors'][0]['location']
                                    log_builder['message'] = res[i]['errors'][0]['message']
                                    messages.append(log_builder)
                            
                            blob = bucket1.blob(filename)
                            blob.upload_from_string(str(messages))
                            message = "Data uploaded to the Bigquery"
                            
                        except Exception as error:
                            print('This was the error: ', error)
                            message = "Error getting logs"
                            
                        try:
                            blob = bucket2.blob(filename)
                            downloaded_blob = blob.download_as_string()
                            log_message = downloaded_blob.decode("utf-8", "ignore")
                            log_message =log_message.split('\n')
                            bad_row = []
                            bad_json = []
                            bad_input_data = {}
                            for element in log_message:
                                element = element.replace("'model-craft-342921:testing.Task3', ",'')
                                element = element.replace('(','')
                                element = element.replace(')','')
                                element = element.replace("'",'"')
                                bad_row.append(element)
                            for index in range(0,len(bad_row)-1):
                                bad_json.append(json.loads(bad_row[index]))
                            bad_input_data['elements'] = bad_json
                            
                        except Exception as error:
                            print('This was the error: ', error)
                            message = "Error setting up messaging"
                            
                            #end = time.time()
                            #Total_time = end - start
                            #string = """UPDATE `model-craft-342921.testing.Statics` SET Completion_time = '{}' WHERE Project_id = '{}'""".format(Total_time, job_name)
                            #query_job = client.query(string)
                            #query_job.result()
                        
                        try:
                            p1 = beam.Pipeline(options=beam_options)
                            events  = (p1 | 'ReadData' >> beam.io.ReadFromText('gs://error_messages4142022/{}'.format(filename))
                                          | 'Process Messaging' >> beam.ParDo(process_message(),bad_input_data)
                                            | 'Store Final Message' beam.io.WriteToText('gs://complete_message_4182022/{}'.format(filename), shard_name_template = "")
                                      )
                            result = p1.run()
                            result.wait_until_finish()

                        except Exception as error:
                            print('This was the error: ', error)
                            message = "Error setting up the error formating pipeline"

                    #If the file is not a json or the csv this will run
                    else:
                        message = "File type is not excepted"
                    #endif
                except Exception as error:
                    print('This was the error: ', error)
                    message = "There was an error in creating the request"
            #endif
        #This will run if the submition is not a file type
        elif 'file' not in request.files:
            message = "There was no file to upload"
        #endif
    #endif

    #this will render the template on the website
    return render_template("front.html", message = message, tables = tables)


In [ ]:
if __name__ == '__main__':
    app.run()